In [34]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Input, Conv1D, LSTM, Dense
from tensorflow.keras.models import Model
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

def split_df(df, window_size, n_series, split=0.75):
    df_as_np = df.to_numpy()
    X = []
    y = []
    for i in range(len(df_as_np)-window_size):
        row = [[a] for a in df_as_np[i:i+window_size]]
        X.append(row)
        label = df_as_np[i+window_size]
        y.append(label)
    X=np.array(X)
    y=np.array(y)
    X=X.reshape(-1,window_size,n_series)
    X_train=X[:int(split*len(X))]
    y_train=y[:int(split*len(X))]
    X_test=X[int(split*len(X)):]
    y_test=y[int(split*len(X)):]

    return X_train,y_train, X_test, y_test


class ConvLSTM(Model):
    def __init__(self, 
                 time_steps, 
                 n_series,
                 forecast_steps,
                 epochs=60,
                 ):
        super(ConvLSTM, self).__init__()
        self.conv1d = Conv1D(32, kernel_size=2)
        self.lstm = LSTM(64)
        self.dense1 = Dense(32, activation='relu')
        self.dense2 = Dense(16, activation='relu')
        self.dense3 = Dense(n_series, activation='linear')
        self.history= None
        self.forecast_setps=forecast_steps
        self.y_test=None
        self.n_series=n_series
        self.epochs=epochs
        self.time_steps=time_steps
        

    def call(self, inputs):
        x = self.conv1d(inputs)
        x = self.lstm(x)
        x = self.dense1(x)
        x = self.dense2(x)
        outputs = self.dense3(x)
        return outputs

    def train(self, X_train, y_train, X_test, y_test):
        self.y_test=y_test
        cp1 = tf.keras.callbacks.ModelCheckpoint('conv_lstm_checkpoint/', save_best_only=True)
        self.compile(loss=tf.keras.losses.MeanSquaredError(), 
                     optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), 
                     metrics=[tf.keras.metrics.RootMeanSquaredError()],
                     )
        self.history = self.fit(X_train, 
                                y_train, 
                                validation_data=(X_test, y_test), 
                                epochs=self.epochs,
                                callbacks=[cp1])
    
    def prediction_summary(self, X_test, summary=False):
        preds = self.predict(X_test)
        if summary==True:
            plt.plot(preds[:100, 0], label='preds')
            plt.plot(self.y_test[:100, 0], label='actual')
            plt.legend()
            plt.show()

            plt.plot(self.history.history['loss'])
            plt.plot(self.history.history['val_loss'])
            plt.title('model loss')
            plt.ylabel('loss')
            plt.xlabel('epoch')
            plt.legend(['train', 'val'], loc='upper right')
            plt.show()
        return preds,self.y_test

    def forecast(self, X_test, forecast_steps, plot_forecast=True):
        last_batch = X_test[-1]
        forecast = []
        for step in range(forecast_steps):
            pred = self.predict(np.array(last_batch).reshape(1, self.time_steps, self.n_series))
            forecast.append(pred)
            last_batch = np.append(last_batch[1:,:], pred, axis=0)
        forecast = np.array(forecast)
        final_forecast = forecast.reshape(forecast_steps, self.n_series)
        if(plot_forecast==True):
            chart_df=pd.DataFrame(np.append(self.y_test[-40:,:], final_forecast, axis=0).reshape(40+forecast_steps,-1))
            chart_df.iloc[:100,1].plot(label='actual')
            chart_df.iloc[100:,1].plot(label='forecast')
            plt.legend()
            plt.show()
            return chart_df
        return final_forecast
        

In [35]:
df.iloc[-700,:]

0     0.578554
1     0.466669
2     0.842208
3     0.560933
4     0.493566
5     0.808935
6     0.706668
7     0.753308
8     0.050264
9     0.648565
10    0.444668
11    0.788406
12    0.658160
13    0.456331
14    0.958175
15    0.583654
16    0.354045
17    0.555175
18    0.611073
19    0.511128
20    0.648392
21    0.716654
22    0.713932
23    0.549496
24    0.537941
25    0.292859
26    0.719034
27    0.703372
28    0.628915
29    0.149350
30    0.525731
31    0.745401
32    0.592261
33    0.272508
34    0.634716
35    0.364415
36    0.628057
37    0.353491
38    0.630781
39    0.733879
40    0.507634
41    0.509659
42    0.365453
43    0.946911
44    0.664385
45    0.715443
46    0.656107
47    0.577320
Name: 2300, dtype: float64

In [36]:
df=pd.read_csv(r"large_portfolio.csv")
df=df.drop('Date', axis=1)
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
df=pd.DataFrame(scaler.fit_transform(df))
X_train,y_train, X_test, y_test=split_df(df=df.iloc[-200:,:6],window_size=14, n_series=6)

In [37]:
time_steps = 14
n_series = 6

conv_lstm_model = ConvLSTM(time_steps, n_series, forecast_steps=15)
conv_lstm_model.train(X_train,y_train, X_test, y_test)

conv_lstm_model.summary()


# Assuming you have test data and forecast_steps: X_test, forecast_steps
final_forecast = conv_lstm_model.forecast(X_test=X_test, forecast_steps=15, plot_forecast=True)
print(final_forecast)

Epoch 1/60


1/5 [=====>........................] - ETA: 7s - loss: 0.4473 - root_mean_squared_error: 0.6688

INFO:tensorflow:Assets written to: conv_lstm_checkpoint\assets


INFO:tensorflow:Assets written to: conv_lstm_checkpoint\assets


5/5 [==============================] - 6s 1s/step - loss: 0.2366 - root_mean_squared_error: 0.4864 - val_loss: 0.0757 - val_root_mean_squared_error: 0.2751
Epoch 2/60
5/5 [==============================] - ETA: 0s - loss: 0.0300 - root_mean_squared_error: 0.1733

...

5/5 [==============================] - 4s 1s/step - loss: 0.0012 - root_mean_squared_error: 0.0347 - val_loss: 0.0038 - val_root_mean_squared_error: 0.0619
Epoch 52/60
5/5 [==============================] - ETA: 0s - loss: 0.0013 - root_mean_squared_error: 0.0355

INFO:tensorflow:Assets written to: conv_lstm_checkpoint\assets


INFO:tensorflow:Assets written to: conv_lstm_checkpoint\assets


5/5 [==============================] - 4s 1s/step - loss: 0.0013 - root_mean_squared_error: 0.0355 - val_loss: 0.0033 - val_root_mean_squared_error: 0.0576
Epoch 53/60
5/5 [==============================] - 0s 30ms/step - loss: 0.0013 - root_mean_squared_error: 0.0367 - val_loss: 0.0058 - val_root_mean_squared_error: 0.0764
Epoch 54/60
5/5 [==============================] - 0s 25ms/step - loss: 0.0011 - root_mean_squared_error: 0.0335 - val_loss: 0.0063 - val_root_mean_squared_error: 0.0793
Epoch 55/60
5/5 [==============================] - 0s 24ms/step - loss: 9.2934e-04 - root_mean_squared_error: 0.0305 - val_loss: 0.0053 - val_root_mean_squared_error: 0.0726
Epoch 56/60
5/5 [==============================] - 0s 24ms/step - loss: 8.7210e-04 - root_mean_squared_error: 0.0295 - val_loss: 0.0048 - val_root_mean_squared_error: 0.0695
Epoch 57/60
5/5 [==============================] - 0s 22ms/step - loss: 8.6519e-04 - root_mean_squared_error: 0.0294 - val_loss: 0.0046 - val_root_mean_squa

ValueError: cannot reshape array of size 330 into shape (115,newaxis)

In [ ]:
y_test[-40:,:].shape

(40, 6)

In [ ]:
pd.DataFrame(X_test[-1][1:,:])

,0,1,2,3,4,5,6,7,8,9,...,38,39,40,41,42,43,44,45,46,47
0,177.899994,122.230003,123.010002,323.940002,140.669998,225.089996,158.500000,145.339996,108.769997,16.040001,...,171.479996,81.885236,512.500000,91.280745,790.697485,132.009995,197.869995,74.480003,173.070007,44.291095
1,181.500000,122.250000,124.080002,324.989990,140.350006,223.639999,160.419998,145.679993,108.459999,16.010000,...,171.000000,82.490465,518.190002,90.886021,801.529758,132.949997,197.630005,74.860001,173.429993,44.467395
2,181.270004,122.790001,124.019997,328.579987,141.070007,224.410004,160.000000,147.000000,105.809998,15.950000,...,170.839996,83.184994,517.280029,91.330092,806.821601,131.960007,198.210007,74.410004,171.529999,44.320476
3,182.800003,124.980003,128.119995,334.470001,140.479996,225.619995,159.729996,144.509995,107.379997,15.800000,...,171.309998,84.504608,522.020020,91.320221,856.566582,134.660004,198.500000,73.900002,176.559998,44.594725
4,183.369995,123.099998,126.699997,334.339996,143.000000,223.740005,161.600006,145.309998,107.449997,15.900000,...,173.660004,87.838357,525.000000,92.741246,851.364321,134.639999,202.000000,74.900002,175.820007,44.900002
5,183.960007,123.139999,125.209999,337.480011,140.589996,222.020004,162.050003,147.169998,105.500000,15.880000,...,175.460007,87.262888,529.000000,92.464932,879.723200,134.919998,201.250000,73.739998,176.240005,44.119999
6,186.729996,125.930000,127.709999,351.320007,143.050003,226.449997,164.479996,148.919998,106.360001,16.100000,...,179.990005,87.590308,530.099976,93.501090,884.567439,139.039993,205.000000,74.750000,179.449997,44.220001
7,184.410004,122.930000,124.970001,339.309998,142.610001,227.000000,164.259995,149.750000,104.680000,16.020000,...,176.460007,87.828431,521.950012,94.497780,858.088455,136.059998,202.649994,75.629997,174.940002,44.029999
8,184.899994,122.400002,125.639999,336.369995,142.899994,226.119995,163.580002,148.259995,102.500000,15.700000,...,174.130005,88.195544,517.909973,93.747798,860.000000,135.570007,199.289993,74.449997,172.500000,43.470001
9,183.740005,120.000000,125.309998,334.119995,142.309998,226.360001,163.899994,150.350006,102.839996,15.570000,...,172.160004,88.010002,520.690002,95.220001,845.090027,134.380005,201.960007,76.070000,170.000000,44.099998


In [61]:
pd.DataFrame(X_test[-1])

,0,1,2,3,4,5,6,7,8,9,...,38,39,40,41,42,43,44,45,46,47
0,178.440002,126.970001,127.010002,331.649994,139.550003,227.000000,158.100006,144.009995,106.790001,15.630000,...,167.350006,83.135394,514.510010,90.649184,785.634520,132.750000,196.960007,73.639999,171.169998,43.801366
1,177.899994,122.230003,123.010002,323.940002,140.669998,225.089996,158.500000,145.339996,108.769997,16.040001,...,171.479996,81.885236,512.500000,91.280745,790.697485,132.009995,197.869995,74.480003,173.070007,44.291095
2,181.500000,122.250000,124.080002,324.989990,140.350006,223.639999,160.419998,145.679993,108.459999,16.010000,...,171.000000,82.490465,518.190002,90.886021,801.529758,132.949997,197.630005,74.860001,173.429993,44.467395
3,181.270004,122.790001,124.019997,328.579987,141.070007,224.410004,160.000000,147.000000,105.809998,15.950000,...,170.839996,83.184994,517.280029,91.330092,806.821601,131.960007,198.210007,74.410004,171.529999,44.320476
4,182.800003,124.980003,128.119995,334.470001,140.479996,225.619995,159.729996,144.509995,107.379997,15.800000,...,171.309998,84.504608,522.020020,91.320221,856.566582,134.660004,198.500000,73.900002,176.559998,44.594725
5,183.369995,123.099998,126.699997,334.339996,143.000000,223.740005,161.600006,145.309998,107.449997,15.900000,...,173.660004,87.838357,525.000000,92.741246,851.364321,134.639999,202.000000,74.900002,175.820007,44.900002
6,183.960007,123.139999,125.209999,337.480011,140.589996,222.020004,162.050003,147.169998,105.500000,15.880000,...,175.460007,87.262888,529.000000,92.464932,879.723200,134.919998,201.250000,73.739998,176.240005,44.119999
7,186.729996,125.930000,127.709999,351.320007,143.050003,226.449997,164.479996,148.919998,106.360001,16.100000,...,179.990005,87.590308,530.099976,93.501090,884.567439,139.039993,205.000000,74.750000,179.449997,44.220001
8,184.410004,122.930000,124.970001,339.309998,142.610001,227.000000,164.259995,149.750000,104.680000,16.020000,...,176.460007,87.828431,521.950012,94.497780,858.088455,136.059998,202.649994,75.629997,174.940002,44.029999
9,184.899994,122.400002,125.639999,336.369995,142.899994,226.119995,163.580002,148.259995,102.500000,15.700000,...,174.130005,88.195544,517.909973,93.747798,860.000000,135.570007,199.289993,74.449997,172.500000,43.470001
